In [1]:
import numpy as np
#from School.STAT946.Project import GAN, Distill_Model
import GANmike, Distill_Model
#from School.STAT946.Project.mnist_challenge.model import Model as targ_model
from keras.datasets import mnist
from keras.utils import to_categorical
from tqdm import tqdm
from keras.models import load_model as load
import tensorflow as tf
from sklearn.metrics import accuracy_score
#from matplotlib import pyplot as plt

import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
from tensorflow.python.client import device_lib
import pickle
print(device_lib.list_local_devices())
#system_path= 'ResearchDocuments/STAT946/Project/'
system_path= './'

def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3040077643807868816
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 4591050474028247531
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 11330676327
locality {
  bus_id: 1
}
incarnation: 1571590385801101956
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 11330614068
locality {
  bus_id: 1
}
incarnation: 13594789490573055085
physical_device_desc: "device: 2, name: Tesla K80, pci bus id: 0000:0e:00.0"
]


In [2]:
outputfile=open('DynDistClog.txt','w')

def train_for_n(batches=20000,batch_size=128,distillation='static'):

    #Distill the adversarial model
    print('Initial distillation (same for static and dynamic)')
    Distill_Model.Distill_Model(target=target, model=f, X=X,iterations=5, mode='epoch')


    for t in range(batches):

        # Make generative images & select real images
        real_image_batch = X[np.random.randint(0,X.shape[0],size=int(batch_size/2)),:,:,:]
        fake_sample=np.random.randint(0,X.shape[0],size=int(batch_size/2))
        fake_image_inp = X[fake_sample,:,:,:]
        fake_image_batch = np.add(fake_image_inp,G.predict(fake_image_inp))

        # Train discriminator on generated images
        X_batch = np.concatenate((real_image_batch, fake_image_batch))
        y1 = np.zeros([batch_size,1])
        y1[0:int(batch_size/2),] = 1

        #train the discriminator
        make_trainable(D,True)
        D.train_on_batch(X_batch,y1)

        #sample more real images to train the generator
        sample_int=np.random.randint(0,X.shape[0],size=int(batch_size))
        fake_image_inp = X[sample_int,:,:,:]
        y_discrim = np.ones([batch_size,1])
        y_class=y[sample_int]
        y_hinge=np.zeros([batch_size,28,28,1])

        make_trainable(D,False)
        make_trainable(f,False)
        h_GAN = GAN_mod.train_on_batch(fake_image_inp, [y_discrim,y_class,y_hinge])

        #Dynamic distillation
        if distillation=='dynamic':
            make_trainable(f,True)
            Distill_Model.Distill_Model(target=target, model=f, X=X_batch,iterations=2,mode='batch')

        if t%100==0:
            targ_pred=np.argmax(target.predict(fake_image_batch),1)
            adv_acc=accuracy_score(np.argmax(y[fake_sample],1),targ_pred)
            outputfile.write('Dynamic Accuracy: ' + str(adv_acc))
            #plt.imshow(fake_img_batch[0,])

    return

In [3]:
#modelname is an uppercase letter
GAN_mod, G, D, f= GANmike.Define_GAN([28,28,1], 100, 10000,'C')
print('Importing target model C')
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,logits=predicted)
target=load('./ModelC_PristineOnly', custom_objects={'fn':fn})


print('Loading MNIST data')
(X,y),(X_test,y_test)=mnist.load_data()
X= np.divide(X,255)
X=X.reshape(X.shape[0],28,28,1)
X_test= np.divide(X_test,255)
X_test=X_test.reshape(X_test.shape[0],28,28,1)
y=to_categorical(y, num_classes=10)
y_test=to_categorical(y_test, num_classes=10)


TypeError: categorical_crossentropy() missing 2 required positional arguments: 'target' and 'output'

In [ ]:
print('Training')
train_for_n(batches=10000,batch_size=128,distillation='dynamic')


In [ ]:
GAN_mod, G, D, f= GANmike.Define_GAN([28,28,1], 100, 10000,'')
GAN_mod.save_weights('./dynamicdistweights/C_DynDist_Ganwt10k')
G.save_weights('./dynamicdistweights/C_DynDist_Gwt10k')
D.save_weights('./dynamicdistweights/C_DynDist_Dwt10k')
f.save_weights('./dynamicdistweights/C_DynDist_fwt10k')

In [ ]:
X_test_perturb=np.add(X_test,G.predict(X_test))
targ_pred=np.argmax(target.predict(X_test_perturb),1)
adv_acc=accuracy_score(np.argmax(y_test,1),targ_pred)
print('Test Accuracy: ' + str(adv_acc))

In [ ]:
#My testing
y_test=pickle.load(open('Y_testm.p','rb'))
prx_test=pickle.load(open('X_testm.p','rb'))
#f2=load('ModelC_PristineOnly', custom_objects={'fn':fn})
#prprd=f2.predict(prx_test)
#advprd=f2.predict(generated_images)
#prprd_y = np.argmax(prprd, axis=1)
#advprd_y = np.argmax(advprd, axis=1)
prscore=target.evaluate(prx_test,y_test, verbose=1)
advscore=target.evaluate(X_test_perturb, y_test, verbose=1)
print('Pristine Images: Test Accuracy:',prscore[1])
print('Adv Images Test accuracy:',advscore[1])


In [ ]:
#GAN_mod.load_weights('./dynamicdistweights/C_DynDist_Ganwt20k')
#G.load_weights('./dynamicdistweights/C_DynDist_Gwt20k')
#D.load_weights('./dynamicdistweights/C_DynDist_Dwt20k')
#f.load_weights('./dynamicdistweights/C_DynDist_fwt20k')